In [ ]:
import helpers.i2c_gui2_helpers as helpers
import datetime, time
import numpy as np
from tqdm import tqdm
import pandas as pd

In [ ]:
chip_names = ["Wafer_N62C72_Die52_R5C5"]
batch, wafer, location = 0,0,52

In [ ]:
def efuse_enable(chip, batch=0, wafer=0, location=0, prog_bypass=None):
    df = pd.read_csv('all_bch_encoded_data.csv')
    hex_str = df[(df["Batch"]==batch) & (df["Wafer"]==wafer) & (df["Location"]==location)]["Word Message"].to_list()[0]
    int_str = int(hex_str, 16)
    print("EFuse Word Message  :", f"{int_str:032b}", hex_str)
    peri_config = {
        "EFuse_EnClk": 0b1,
        "EFuse_Bypass": 0b0,
        "EFuse_Rstn": 0b1,
        "EFuse_Start": 0b0,
        "EFuse_Mode": 0b10,
        "EFuse_TCKHP": 0x4,
        "EFuse_Prog": int_str if (prog_bypass==None) else prog_bypass,
    }
    chip.read_all_block("ETROC2", "Peripheral Config")
    for key, value in peri_config.items():
        chip.set_decoded_value("ETROC2", "Peripheral Config", key, value)
    chip.write_all_block("ETROC2", "Peripheral Config")

In [ ]:
def efuse_disable(chip):
    peri_config = {
        "EFuse_EnClk": 0b0,
        "EFuse_Bypass": 0b1,
        "EFuse_Mode": 0b10,
        "EFuse_TCKHP": 0x4,
    }
    chip.read_all_block("ETROC2", "Peripheral Config")
    for key, value in peri_config.items():
        chip.set_decoded_value("ETROC2", "Peripheral Config", key, value)
    chip.write_all_block("ETROC2", "Peripheral Config")

In [ ]:
def efuse_reset(chip):
    chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Rstn')
    chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Rstn', 0b0)
    chip.write_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Rstn')
    chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Rstn', 0b1)
    chip.write_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Rstn')

In [ ]:
def efuse_read(chip):
    peri_config = [
    "EFuse_EnClk",
    "EFuse_Bypass",
    "EFuse_Rstn",
    "EFuse_Start",
    "EFuse_Mode",
    "EFuse_TCKHP",
    "EFuse_Prog",
    ]
    # Read Mode
    chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Mode')
    chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Mode', 0b10)
    chip.write_decoded_value("ETROC2", "Peripheral Config", "EFuse_Mode")
    # Reset
    # efuse_reset(chip)

    chip.read_all_block("ETROC2", "Peripheral Config")
    # Read all Registers
    for key in peri_config:
        value = chip.get_decoded_value("ETROC2", "Peripheral Config", key)
        print(key, value)
        if(key=='EFuse_Prog'):
            print("Chip ID (EF Prog [LSB 17 bits])- ",f"{(value & 0x0000ffff):#06x}",f"{(value & 0x0000ffff):>017b}")
            print(f"EFProg Intd for chip: {value:032b}, {hex(value)}, {value}")
    # Read EFuseQ
    chip.read_decoded_value("ETROC2", "Peripheral Status", 'EFuseQ')
    EFuseQ = chip.get_decoded_value("ETROC2", "Peripheral Status", "EFuseQ")
    print(f"EFuseQ Read for chip: {EFuseQ:032b}, {hex(EFuseQ)}, {EFuseQ}")

In [ ]:
def efuse_check_write(chip):
    # Read Mode
    chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Mode')
    chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Mode', 0b10)
    chip.write_decoded_value("ETROC2", "Peripheral Config", "EFuse_Mode")
    # Reset
    # efuse_reset(chip)
    # Read EFuse Prog and EFuseQ
    chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Prog')
    EFuse_Prog = chip.get_decoded_value("ETROC2", "Peripheral Config", "EFuse_Prog")
    chip.read_decoded_value("ETROC2", "Peripheral Status", 'EFuseQ')
    EFuseQ = chip.get_decoded_value("ETROC2", "Peripheral Status", "EFuseQ")
    if(EFuse_Prog==EFuseQ): return True
    else: return False

In [ ]:
port = "/dev/ttyACM4"
chip_addresses = [0x60]
ws_addresses = [None]
i2c_conn = helpers.i2c_connection(port,chip_addresses,ws_addresses,chip_names)
for chip_address in chip_addresses[:]:
    i2c_conn.calibratePLL(chip_address, chip=None)
for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)

In [ ]:
# i2c_conn.config_chips(
#     do_pixel_check=False,
#     do_basic_peripheral_register_check=False,
#     do_disable_all_pixels=True,
#     do_auto_calibration=False,
#     do_disable_and_calibration=False,
#     do_prepare_ws_testing=False
# )

In [ ]:
# 0b00010010110101000000000100000000

In [ ]:
chip_address = chip_addresses[0]
chip = i2c_conn.get_chip_i2c_connection(chip_address)
efuse_enable(chip, batch, wafer, location, prog_bypass=None)
efuse_reset(chip)
efuse_read(chip)

In [ ]:
for pw in range(1):

    for i in tqdm(range(1000)):
        # Reset
        # efuse_reset(chip)

        # Write Mode
        chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Mode')
        chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Mode', 0b01)
        chip.write_decoded_value("ETROC2", "Peripheral Config", "EFuse_Mode")

        # Reset
        efuse_reset(chip)

        for run_count in range(1):
            chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Start')
            chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Start', 0b1)
            chip.write_decoded_value("ETROC2", "Peripheral Config", "EFuse_Start")
            # time.sleep(5)
            # chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Start')
            chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Start', 0b0)
            chip.write_decoded_value("ETROC2", "Peripheral Config", "EFuse_Start")

        if(efuse_check_write(chip)): 
            print(i, "SUCCESS")
            break
    else:
        # time.sleep(1)
        # efuse_disable(chip)
        # time.sleep(1)
        # efuse_enable(chip, batch, wafer, location, prog_bypass=0b00010011111111111111111111111111)
        # efuse_reset(chip)
        continue
    break

In [ ]:
for pw in [4, 12, 15]:
    # Pulse Width
    print(f"-----{pw}-----")
    chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_TCKHP')
    chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_TCKHP', pw)
    chip.write_decoded_value("ETROC2", "Peripheral Config", "EFuse_TCKHP")

    for i in tqdm(range(500)):
        # Reset
        # efuse_reset(chip)

        # Write Mode
        chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Mode')
        chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Mode', 0b01)
        chip.write_decoded_value("ETROC2", "Peripheral Config", "EFuse_Mode")

        # Reset
        efuse_reset(chip)

        for run_count in range(1):
            chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Start')
            chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Start', 0b1)
            chip.write_decoded_value("ETROC2", "Peripheral Config", "EFuse_Start")
            # time.sleep(5)
            # chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Start')
            chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_Start', 0b0)
            chip.write_decoded_value("ETROC2", "Peripheral Config", "EFuse_Start")

        if(efuse_check_write(chip)): 
            print(i, "SUCCESS")
            break
    else:
        # time.sleep(1)
        # efuse_disable(chip)
        # time.sleep(1)
        # efuse_enable(chip, batch, wafer, location, prog_bypass=0b00010011111111111111111111111111)
        # efuse_reset(chip)
        continue
    break

In [ ]:
# chip.read_decoded_value("ETROC2", "Peripheral Config", 'EFuse_EnClk')
# chip.set_decoded_value("ETROC2", "Peripheral Config", 'EFuse_EnClk', 0b1)
# chip.write_decoded_value("ETROC2", "Peripheral Config", "EFuse_EnClk")
efuse_read(chip)
efuse_disable(chip)